In [ ]:
!pip install transformers

In [ ]:
!pip install -U sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

In [ ]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
# model_path =
# answer_path =

In [ ]:
import random
import os

import pandas as pd
import numpy as np

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42


In [ ]:
train= pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')
submission = pd.read_csv(f'{data_path}sample_submission.csv')

train_ft = pd.read_csv(f'{data_path}category_train_80000_ver3.csv')


# Kogpt2

In [ ]:
import torch
import transformers
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)


In [ ]:
# text = '실내장식이란?'
# input_ids = tokenizer.encode(text, return_tensors='pt')
# gen_ids = model.generate(input_ids,
#                            max_length=256,
#                            repetition_penalty=2.0,
#                            pad_token_id=tokenizer.pad_token_id,
#                            eos_token_id=tokenizer.eos_token_id,
#                            bos_token_id=tokenizer.bos_token_id,
#                            use_cache=True)
# generated = tokenizer.decode(gen_ids[0])
# print(generated)

실내장식이란?.
이런저렇게 다양한 종류의 장식들이 있는데요.
그중에서도 가장 눈에 띄는 것은 바로 이게 뭐냐 하면 그~ 천장에 있는 커다란 유리창에 달린 거울입니다.
거울을 보면은 마치 한 폭의 그림 같은데요?
네. 네. 그래서 이렇게 작은 창으로 된 거죠.
아하 예예쁘네요라고 하는 건가 봐요
어우 진짜로 어머나 신기한 게 있죠?
뭐 이런 것도 있고 또 하나는 이제 벽면에 붙어있는 여러 가지 소품들인데 아까 말씀드린 것처럼 굉장히 많은 것들이 있습니다.
그래서 그런 것들을 통해서 좀 더 쉽게 볼 수 있도록 해서 보는 재미도 쏠쏠하게 할 수가 있겠습니다.
자 그럼 오늘 소개해 드릴 세 번째 시간에는 어떤 주제로 이야기를 나눠볼까요?
오늘 첫 번째는 우리 주부님들을 위한 선물 고르는 법부터 살펴보겠는데 먼저 우리가 흔히 생각하는 선물은 무엇일까요?
바로 뭘까?
선물이라고 하기에는 너무 비싸고 비싼 것 같아 가지고 많이 사야 되잖습니까.
그러니까 선물을 구입하는 데 있어서 꼭 필요한 것이 무엇인지 알아보고 구매를 결정하는 방법 중에 하나가 무엇이 있을지 한번 생각해 보도록 하겠습니다. 고맙구요.</d> 요즘 날씨가 점점 추워지고 있어요,
내일 아침 출근길에도 춥다고 하는데 내일은 영하로 떨어집니다.
서울


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        question = item['question']
        answer = item['answer']
        category = item['category']
        qna_category = item['qna_category']
        return {'question': question, 'answer': answer, 'category': category, 'qna_category':qna_category}

In [ ]:
def collate_fn(batch):
    texts = [f"<q>{item['question']}</s><a>{item['answer']}</s><c>{item['category']}</c><qnc>{item['qna_category']}</qnc>"
    for item in batch]

    x = tokenizer(texts, return_tensors='pt', padding=True)
    return {'x':x}

In [ ]:
dt = ChatDataset(train_ft)
dt[0]

{'question': '면진장치가 뭐야?',
 'answer': '면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.',
 'category': "['건축구조']",
 'qna_category': "['설명']"}

In [ ]:
dt = ChatDataset(train_ft)
dt[45000]

{'question': '철골구조의 장점이 뭐야? 또한, 장판이 남으면 어떻게 처리해?',
 'answer': '철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다. 또한, 장판 처리방법은 구청, 주민센터에서 생활폐기물 스티커를 구매 후 배출해야 합니다. 단, 장판은 크기마다 비용이 다르므로 사전에 크기를 확인해야 합니다.',
 'category': "['건축구조', '마감재']",
 'qna_category': "['장점', '방법']"}

In [ ]:
dl = torch.utils.data.DataLoader(dt, batch_size=2, collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405,  7532,  8265, 37765, 46651,  7991,   406,     1,
           9724,   439,   405,  7532,  8265, 20725,  7374,  9027,  7599,  9023,
          14472, 15898, 14820, 33220, 36928, 10764,  9166, 11818, 28037, 10090,
          15898, 34062, 20725, 21154,     1,  9724,   441,   405,   433,   382,
          13736, 12791,   382,   435,   403,   390,   441,   405,   403,   455,
            452,   441,   405,   433,   382, 16623,   382,   435,   403,   390,
            455,   452,   441,   405],
         [ 9724,   455,   405,  7071,  8265,  7793, 10346, 10036, 11732,  9666,
           8244,     1,  9724,   439,   405,  7071,  8265,  9160, 10346, 32032,
           9094,  8265, 12791,   387,  9037,  8265, 12791,   387,  9411,  8265,
          27211, 32987,     1,  9724,   441,   405,   433,   382, 13736, 12791,
            382,   435,   403,   390,   441,   405,   403,   455,   452,   441,
            405,   433,   382, 13021,   382,   435,   403,   3

In [ ]:
model(**batch['x']).logits.shape


torch.Size([2, 64, 51200])

# Train Loop

In [ ]:
len(max(train_ft['answer']))

236

In [ ]:
# reset_seeds(SEED)

from tqdm import tqdm

def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch['x'].to(device) # 학습을 위해서 device로 옮김

        pred = model(**x).logits # batch, seq, 단어별
        n_class = pred.shape[-1] # kogpt2 출력수 51200

        pred = pred[:, :-1, :] # 마지막 차원의 마지막 토큰인 eos 토큰만 제외함
        pred = pred.reshape(-1, n_class) # 2차원 형태로 변환

        tgt = x['input_ids'][:,1:]
        tgt = tgt.flatten() # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id # 손실 계산시 제외
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred, tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [ ]:

reset_seeds(SEED)

batch_size = 2
epochs = 10

from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(train_ft)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")
    model.save_pretrained(f"{data_path}models/category_kogpt2_trial1_epoch{i+1}_loss{train_loss:.4f}")
    torch.cuda.empty_cache()

# 14번째 스탑

100%|██████████| 40000/40000 [1:29:46<00:00,  7.43it/s]


1번째 epoch: 0.4359


100%|██████████| 40000/40000 [1:30:12<00:00,  7.39it/s]


2번째 epoch: 0.2332


100%|██████████| 40000/40000 [1:30:01<00:00,  7.41it/s]


...

100%|██████████| 40000/40000 [1:29:05<00:00,  7.48it/s]


5번째 epoch: 0.1444


100%|██████████| 40000/40000 [1:28:41<00:00,  7.52it/s]


6번째 epoch: 0.1366


  2%|▏         | 825/40000 [01:49<1:18:10,  8.35it/s]